## Summary Data for GraphML Files

In [11]:
import graph_tool.all as gt
import os
import numpy as np
import graph_tool
from tqdm import tqdm_notebook

from lib.analysis import clustering
from lib.analysis import dist
from lib.analysis import model
from lib.analysis import power
from lib.analysis import stats

## Part 1: Summary Statistics

Basic statistics for each country's government

In [2]:
data = 'data'
ignore = ['.DS_Store', 'us']
nets = {}

In [71]:
for country in [d for d in os.listdir(data) if d not in ignore]:
    path = os.path.join(data, country)
    if country not in nets:
        nets[country] = []
    for dir_ in [f for f in os.listdir(path) if f not in ignore]:
        if not dir_.endswith('.graphml'):
            net_path = os.path.join(path, dir_)
            for net in [f for f in os.listdir(net_path) if f not in ignore]:
                if net.endswith('.graphml'):
                    nets[country].append((net[:-8], gt.load_graph(os.path.join(net_path, net))))

In [72]:
for country, graphs in nets.items():
    with open(f'summary.{country}.csv', 'w+') as f:
        f.write(','.join(['Number of Edges', 'Number of Vertices',
                  'Clustering Coefficient', 'Average Degree',
                  'Average Excess Degree', 'Average Distance',
                  'Size of LCC', 'Number of CC']) + '\n')
        for name, graph in graphs:
            f.write(f'{name},{str(stats.summary(graph))}\n')

## Part 2: Assortivity

Computes assoritivty on specified attribute for every country across years 

In [5]:
individual_graphs = {}
root = 'data/'
countries = os.listdir(root)
nets = [country + "/" + net + "/" for country in countries
                            for net in os.listdir(root + country)
                            if not net.endswith('.graphml')]
for net in nets:
    for x in os.listdir(root+net):
        if x.endswith('.graphml'):
            individual_graphs[x] = gt.load_graph(root + net + x) 

In [13]:
attribute = 'part'
with open("assortivity_summary.csv", "w+") as f:
    f.write(','.join(['Assortivity Measure']) + '\n')
    for name, g in individual_graphs.items():
        if attribute in g.vertex_properties:
            print(name, attribute)
            g.set_directed(False)
            gt.remove_parallel_edges(g)
            stats.generate_party_property(g, attribute)
#             print(g.vertex_properties[attribute])
            print(gt.assortativity(g, g.vertex_properties[attribute]))
#             print(stats.assortativity_summary(g, attribute))
#             f.write(f'{name},{str(stats.assortativity_summary(g, attribute))}\n')


net_se2014-2018.graphml party <class 'str'>
net_se2010-2014.graphml party <class 'str'>
net_se2006-2010.graphml party <class 'str'>
net_se1998-2002.graphml party <class 'str'>
net_se2002-2006.graphml party <class 'str'>
net_se1988-1991.graphml party <class 'str'>
net_se1994-1998.graphml party <class 'str'>
net_se1991-1994.graphml party <class 'str'>
net_fr_an2007-2012.graphml party <class 'str'>
net_fr_an2002-2007.graphml party <class 'str'>
net_fr_an1986-1988.graphml party <class 'str'>
net_fr_an2012-2017.graphml party <class 'str'>
net_fr_an1997-2002.graphml party <class 'str'>
net_fr_se1997-2002.graphml party <class 'str'>
net_fr_se2012-2017.graphml party <class 'str'>
net_fr_se1986-1988.graphml party <class 'str'>
net_fr_se1988-1993.graphml party <class 'str'>
net_fr_se1993-1997.graphml party <class 'str'>
net_fr_se2002-2007.graphml party <class 'str'>
net_fr_se2007-2012.graphml party <class 'str'>
net_ch_cs1995-1999.graphml party <class 'str'>
net_ch_cs2003-2007.graphml party <cla